In [1]:
# importing required modules
import pandas as pd
import gmaps
import requests
from config import gmap_api_key

# Reading API Keys for Google Map API
gmaps.configure(api_key=gmap_api_key)

In [2]:
# Loading weather data
city_data_df = pd.read_csv("data/WeatherPy_challenge.csv")

# Ask the customer to add a min temp, a max temp and a preference for weather
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no)")
snowing = input("Do you want it to be snowing? (yes/no)")

What is the minimum temperature you would like for your trip?  80
What is the maximum temperature you would like for your trip?  90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [3]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if raining == 'yes' and snowing == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df['Rain inches (last 3hrs)'] > 0) | (preferred_cities_df['Snow inches (last 3hrs)'] > 0)]
elif raining == 'yes' and snowing == 'no':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df['Rain inches (last 3hrs)'] > 0)]
elif raining == 'no' and snowing == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df['Snow inches (last 3hrs)'] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df['Rain inches (last 3hrs)'] == 0) | (preferred_cities_df['Snow inches (last 3hrs)'] == 0)]

In [4]:
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                    118
City                       118
Country                    118
Date                       118
Lat                        118
Lng                        118
Max Temp                   118
Humidity                   118
Cloudiness                 118
Wind Speed                 118
Current Description        118
Rain inches (last 3hrs)    118
Snow inches (last 3hrs)    118
dtype: int64

In [5]:
# Create DataFrame called hotel_df to store hotel name along with city, country, max temp and coordinate
hotel_df = preferred_cities_df[['City','Country','Max Temp','Current Description','Lat','Lng']].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Karak,JO,86.00,haze,31.17,35.75,
14,Grand Gaube,MU,82.99,broken clouds,-20.01,57.66,
28,Bambous Virieux,MU,82.99,broken clouds,-20.34,57.76,
35,Lubao,PH,86.00,few clouds,14.92,120.55,
37,Touros,BR,86.00,broken clouds,-5.20,-35.46,
38,Dwarka,IN,80.47,clear sky,22.24,68.97,
44,San Carlos,VE,82.15,overcast clouds,9.66,-68.58,
52,Talavera,PH,84.20,few clouds,15.62,120.93,
54,Kasane,BW,86.00,scattered clouds,-17.82,25.15,
64,Anloga,GH,89.60,few clouds,5.79,0.90,


In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_api_key
}

In [7]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"{row['City']} : Hotel not found... skipping.")

print("----------------------------------------")
print("Data Retrieval Completed")
print("----------------------------------------")

Micheweni : Hotel not found... skipping.
Hobyo : Hotel not found... skipping.
Thinadhoo : Hotel not found... skipping.
Bondo : Hotel not found... skipping.
Beloha : Hotel not found... skipping.
Tamasane : Hotel not found... skipping.
Marovoay : Hotel not found... skipping.
Izazi : Hotel not found... skipping.
Madimba : Hotel not found... skipping.
Ormara : Hotel not found... skipping.
Bonfim : Hotel not found... skipping.
----------------------------------------
Data Retrieval Completed
----------------------------------------


In [13]:
# Create the output file (csv)
output_data_file = "data/WeatherPy_vacation.csv"
# Export the DataFrame into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# Creating information to display with pop-up marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [11]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [12]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))